In [34]:
#!/usr/bin/env python3

import decimal
import sys
import gurobipy as g
import numpy as np
from itertools import permutations, combinations

file_path = 'instances/cat_part.txt'

f = open(file_path)

data = []

for line in f:
    data_line = []
    for j in line.split(' '):
        data_line.append(j)
    data.append(data_line)


def distance(A, B):
    A = np.array(A[:, -3:])
    B = np.array(B[:, :3])
    return np.sum(np.absolute(A - B))

In [35]:
def find_one_tour(edges):
    tour = []
    tour.append(edges[0])
    start = edges[0][0]
    temp = edges[0][1]
    edges.remove(tour[0])

    while True:
        for edge in edges:
            if edge[0] == temp:
                tour.append(edge)
                if edge[1] == start:
                    edges.remove(edge)
                    return tour, edges
                else:
                    temp = edge[1]
                    edges.remove(edge)
                    break

                    
def subtour(edges):
    shortest_tour = edges[:]

    while len(edges) > 1:
        tour, edges = find_one_tour(edges)
        if len(tour) < len(shortest_tour):
            shortest_tour = tour
    return shortest_tour


def subtourelim(model, where):
  if where == g.GRB.callback.MIPSOL:
    vals = model.cbGetSolution(model._vars)
    selected = g.tuplelist((i, j) for i, j in model._vars.keys()if vals[i, j] > 0.5)

    tour = subtour(selected)
    if len(tour) < n:
      expr = 0
      for i in range(len(tour)):
          expr += model._vars[tour[i][0], tour[i][1]]
      model.cbLazy(expr <= len(tour)-1)

In [36]:
n = int(data[0][0])
w = int(data[0][1])
h = int(data[0][2])

picture = []
piece = []

for i in range(0, n):
    piece = np.asarray(data[i+1])
    piece = np.resize(piece, (h, w*3))
    picture.append(piece)

picture = np.array(picture)
picture = picture.astype(int)

m = g.Model()

vars = {}

for p in permutations(np.arange(n), 2):
    dist = distance(picture[p[0]], picture[p[1]])
    vars[p[0], p[1]] = m.addVar(obj=dist, vtype=g.GRB.BINARY, name=str(p[0]) + '-' + str(p[1]))

for i in range(n):
    vars[i, i] = m.addVar(obj=0, ub=0, vtype=g.GRB.BINARY, name=str(i) + '-' + str(i))
    vars[n, i] = m.addVar(obj=0, vtype=g.GRB.BINARY, name=str(n) + '-' + str(i))
    vars[i, n] = m.addVar(obj=0, vtype=g.GRB.BINARY, name=str(i) + '-' + str(n))
    
vars[n, n] = m.addVar(obj=0, ub=0, vtype=g.GRB.BINARY, name=str(n) + '-' + str(n))
m.update()


for j in range(n + 1):
    m.addConstr(g.quicksum(vars[i, j] for i in range(n + 1)) == 1)

for i in range(n + 1):
    m.addConstr(g.quicksum(vars[i, j] for j in range(n + 1)) == 1)

m._vars = vars
m.params.LazyConstraints = 1
m.optimize(subtourelim)
m.write('clean2.lp')


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 18 rows, 81 columns and 162 nonzeros
Model fingerprint: 0x8593956c
Variable types: 0 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 452020.00000
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 18 rows, 72 columns, 144 nonzeros
Variable types: 0 continuous, 72 integer (72 binary)

Root relaxation: objective 1.153830e+05, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 123221.000    0    6 452020.000 123221.000  72.7%     -    0s


[(0, 1), (1, 2), (2, 3), (4, 0), (5, 6), (6, 7), (7, 4), (3, 8), (8, 5)]
8 7 6 5 4 3 2 1 

In [38]:
path = []

for i in vars:
    if(vars[i].X == 1):
        path.append(i)

temp = n

for i in path:
    if (i[0] == temp):
        temp = i[1]

while temp != n:
    for i in path:
        if(i[0] == temp):
            print(temp+1, end=" ")
            temp = i[1]
            break

6 7 8 5 1 2 3 4 